In [1]:
import json
import numpy as np
from monty.json import MontyDecoder,MontyEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error
import pandas as pd

In [2]:
with open("Ti_data_both.json") as f:
    data=json.loads(f.read(),cls=MontyDecoder)

In [3]:
bd=-47.039394/6
b1=-47.02055184878181/6

In [32]:
colors=['black','dimgray','lightcoral','brown','red','orangered','chocolate',
        'burlywood','orange','darkgoldenrod','gold','olive','darkolivegreen','chartreuse',
        'green','turquoise','darkcyan','deepskyblue','dodgerblue','midnightblue','blue',
        'blueviolet','violet','deeppink']
index=np.array([0,4,8,12,16,20,1,5,9,13,17,21,2,6,10,14,18,22,3,7,11,15,19,23])
colors_=[]
for i in index:
    colors_.append(colors[i])
colors=colors_
markers=['.','o','v','^','<','>','s','*','D','1','+','x',
            '.','o','v','^','<','>','s','*','D','1','+','x']

# Vacancy

In [84]:
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation'].startswith("vacancies") and data[i]['metadata']['proto']=='mp-865373POSCAR' and data[i]['calc']=='final':
        print(data[i]['metadata'])

{'method': 'vacancy', 'proto': 'mp-865373POSCAR', 'deleted_site': 0, 'perturbation': 'vacancies', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/va_calculations/mp-865373POSCAR/0'}
{'method': 'vacancy', 'proto': 'mp-865373POSCAR', 'deleted_site': 18, 'perturbation': 'vacancies', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/va_calculations/mp-865373POSCAR/1'}


In [85]:
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation'].startswith("vacancies") and data[i]['metadata']['proto']=='casi-alth' and data[i]['calc']=='final':
        print(data[i]['metadata'])

{'method': 'vacancy', 'proto': 'casi-alth', 'deleted_site': 0, 'perturbation': 'vacancies', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/va_calculations/casi-alth/0'}
{'method': 'vacancy', 'proto': 'casi-alth', 'deleted_site': 16, 'perturbation': 'vacancies', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/va_calculations/casi-alth/1'}


In [86]:
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation'].startswith("vacancies") and data[i]['metadata']['proto']=='omega' and data[i]['calc']=='final':
        print(data[i]['metadata'])

{'method': 'vacancy', 'proto': 'omega', 'deleted_site': 0, 'perturbation': 'vacancies', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/va_calculations/omega/0'}
{'method': 'vacancy', 'proto': 'omega', 'deleted_site': 72, 'perturbation': 'vacancies', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/va_calculations/omega/1'}
{'method': 'vacancy_ideal', 'proto': 'omega', 'perturbation': 'vacancies_ideal', 'calc_loc': '/scratch/anirudhrn/titanium/vacancies/ideal_calculations/omega'}


In [87]:
vacancy_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="vacancies" and data[i]['calc']=='final' and not data[i]['metadata']['proto']=='casi-alth' and not data[i]['metadata']['proto']=='mp-865373POSCAR':
        vacancy_id.append(i)
vacancy_ideal_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="vacancies_ideal" and data[i]['calc']=='final':
        vacancy_ideal_id.append(i)

In [88]:
protos=[]
protos_=[]
for id in vacancy_id:
    protos.append(data[id]['metadata']['proto']+'_'+str(data[id]['metadata']['deleted_site'])+'|'+str(id))
    protos_.append(data[id]['metadata']['proto']+'_'+str(data[id]['metadata']['deleted_site']))

In [89]:
protos_ideal={}
for id in vacancy_ideal_id:
    protos_ideal[data[id]['metadata']['proto']]=id

In [90]:
protos_ideal

{'mp-568714POSCAR': 12767,
 'mp-568087POSCAR': 12768,
 'omega': 12769,
 'mp-753POSCAR': 12770,
 'mp-10021POSCAR': 12771,
 'mp-605790POSCAR': 12772,
 'mp-29647POSCAR': 12773,
 'mp-23313POSCAR': 12774,
 'lisi': 12775,
 'crsi2': 12776,
 'tisi2': 12777,
 'mp-639736POSCAR': 12778,
 'ptte': 12779,
 'mp-84POSCAR': 12780,
 'ABAC': 12781,
 'mob': 12782,
 'bcc': 12783,
 'la2sb': 12784,
 'mp-142POSCAR': 12785,
 'bapb3': 12786}

In [91]:
deltaEs=[]
deltaEs_pred=[]
for proto in list(protos):
    delid=int(proto.split('|')[1])
    # try:
    idealid=protos_ideal[proto.split('_')[0]]
    # except:
    #     print(proto.split('_')[0])
        # continue
    N_proto=len(data[idealid]['structure'].species)
    Eideal=data[idealid]['energy']
    Evac=data[delid]['energy']
    Eideal_pred=data[idealid]['2021']['energy']
    Evac_pred=data[delid]['2021']['energy']   
    deltaE=Evac-(N_proto-1)*Eideal/N_proto
    deltaE_pred=Evac_pred-(N_proto-1)*Eideal_pred/N_proto
    deltaEs.append(deltaE)
    deltaEs_pred.append(deltaE_pred)


In [92]:
d={'Structure_site':protos_,r'$E^{DFT} (eV)$':np.round(np.array(deltaEs),5),r'$E^{pred} (eV)$':np.round(np.array(deltaEs_pred),5)}
df = pd.DataFrame(data=d)

In [93]:
df

,Structure_site,$E^{DFT} (eV)$,$E^{pred} (eV)$
0,mp-568714POSCAR_0,0.51703,1.03058
1,omega_0,1.53842,1.79087
2,omega_72,3.05076,3.67771
3,mp-753POSCAR_0,2.07613,2.56099
4,mp-10021POSCAR_0,2.08458,1.54106
5,mp-605790POSCAR_0,0.44979,-3.31487
6,mp-29647POSCAR_0,2.41890,2.08143
7,mp-29647POSCAR_16,1.60607,1.22427
8,mp-23313POSCAR_0,1.95050,1.73306
9,lisi_0,1.96084,1.93382


In [94]:
from pandas.plotting import  table
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(5, 6))
ax = fig.add_subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, df, loc='center')  # where df is your data frame

plt.savefig('Vacancy_formation.png',dpi=200)

# Grain Boundary

In [73]:
gb_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="gb" and data[i]['calc']=='final':
        gb_id.append(i)
icsd_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="icsd" and data[i]['calc']=='final':# and data[i]['metadata']['proto']=='bcc.vasp' or data[i]['metadata']['proto']=='hcp.vasp':
        icsd_id.append(i)

In [75]:
for id in gb_id:
    print(data[id]['metadata']['ref_elem'])

Mo
Mo
Mo
Mo
Mo
Mo
Mo
Ti


In [76]:
deltaEs_gb=[]
deltaEspred_gb=[]
for id in gb_id:
    protoid=0
    if data[id]['metadata']['ref_elem']=='Mo':
        protoid=1000
    if data[id]['metadata']['ref_elem']=='Ti':
        protoid=928
    N_gb=len(data[id]['structure'].species)
    M_icsd=len(data[protoid]['structure'].species)
    E_gb=data[id]['energy']
    E_gb_pred=data[id]['2021']['energy']
    E_icsd=data[protoid]['energy']
    lattice=data[id]['structure'].lattice.matrix
    A=np.linalg.norm(np.cross(lattice[0,:],lattice[1,:]))
    deltaE_gb=(E_gb-E_icsd*N_gb/M_icsd)/2/A
    deltaEpred_gb=(E_gb_pred-E_icsd*N_gb/M_icsd)/2/A
    deltaEs_gb.append(deltaE_gb)
    deltaEspred_gb.append(deltaEpred_gb)


In [77]:
protos=['bcc','bcc','bcc','bcc','bcc','bcc','bcc','hcp']
planes=[data[gb_id[i]]['metadata']['plane'] for i in np.arange(8)]
sigmas=[data[gb_id[i]]['metadata']['sigma'] for i in np.arange(8)]

In [82]:
d={'proto':protos,'plane':planes,'sigma':sigmas,r'$E^{DFT}(eV/Å^{2})$':np.round(np.array(deltaEs_gb),5),r'$E^{pred}(eV/Å^{2})$':np.round(np.array(deltaEspred_gb),5)}
df = pd.DataFrame(data=d)

In [83]:
from pandas.plotting import  table
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(5, 6))
ax = fig.add_subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, df, loc='center')  # where df is your data frame

plt.savefig('Grain_boundary.png',dpi=200)

# Surface

In [99]:
surface_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="surfaces" and data[i]['calc']=='final':
        surface_id.append(i)
icsd_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="icsd" and data[i]['calc']=='final':# and data[i]['metadata']['proto']=='bcc.vasp' or data[i]['metadata']['proto']=='hcp.vasp':
        icsd_id.append(i)
protoids={'hcp.vasp':928,'bcc.vasp':1000,'fcc.vasp':1565,'omega.vasp':900}

In [100]:
deltaEs_sf=[]
deltaEspred_sf=[]
for id in surface_id:
    protoid=protoids[data[id]['metadata']['proto']]
    N_sf=len(data[id]['structure'].species)
    M_icsd=len(data[protoid]['structure'].species)
    E_sf=data[id]['energy']
    E_sf_pred=data[id]['2021']['energy']
    E_icsd=data[protoid]['energy']
    lattice=data[id]['structure'].lattice.matrix
    A=np.linalg.norm(np.cross(lattice[0,:],lattice[1,:]))
    deltaE_sf=(E_sf-E_icsd*N_sf/M_icsd)/2/A
    deltaEpred_sf=(E_sf_pred-E_icsd*N_sf/M_icsd)/2/A
    deltaEs_sf.append(deltaE_sf)
    deltaEspred_sf.append(deltaEpred_sf)


In [105]:
protos=[data[id]['metadata']['proto'].split('.')[0] for id in surface_id]

In [103]:
data[surface_id[0]]['metadata']

{'method': 'surface',
 'miller': [1, 1, 0],
 'ouc': False,
 'proto': 'omega.vasp',
 'perturbation': 'surfaces',
 'calc_loc': '/scratch/anirudhrn/titanium/surfaces/calculations/omega/1_1_0/0'}

In [104]:
protos=[data[id]['metadata']['proto'].split('.')[0] for id in surface_id]
planes=[data[id]['metadata']['miller'] for id in surface_id]

In [106]:
d={'proto':protos,'plane':planes,r'$E^{DFT}(eV/Å^{2})$':np.round(np.array(deltaEs_sf),5),r'$E^{pred}(eV/Å^{2})$':np.round(np.array(deltaEspred_sf),5)}
df = pd.DataFrame(data=d)

In [107]:
from pandas.plotting import  table
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(5, 6))
ax = fig.add_subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, df, loc='center')  # where df is your data frame

plt.savefig('Surfaces.png',dpi=200)

# Strain

In [22]:
protos=set()
for id in strain_id:
    protos.add(data[id]['metadata']['proto'])
protos=list(protos)

In [50]:
strain_id=[]
for i in np.arange(len(data)):
    if data[i]['metadata']['perturbation']=="strain" and data[i]['calc']=='final':
        strain_id.append(i)

In [36]:
for id in strain_id:
    if not 'magnitude' in data[id]['metadata'].keys():
        print(id)

In [60]:
from matplotlib import pyplot as plt
for proto in protos:
    strain_id=[]
    for i in np.arange(len(data)):
        if data[i]['metadata']['perturbation']=="strain" and data[i]['calc']=='final' and data[i]['metadata']['proto']==proto:
            strain_id.append(i)
    Es=np.zeros((len(strain_id),6))
    i=0
    for id in strain_id:
        Es[i]=np.array(data[id]['metadata']['special_direction'])
        i+=1
    groups=[]
    already=np.zeros(len(Es))
    isidentical=abs(np.dot(Es,Es.T)-1)<1e-12
    for i in np.arange(len(Es)):
        if not already[i]==1:
            thisgroup=np.where(isidentical[i]==True)
            groups.append(thisgroup[0].tolist())
            already[thisgroup]=1
    i=0
    with plt.style.context('science'):
        plt.figure() 
        for group in groups:
            magnitudes=[]
            Energies=[]
            Energies_pred=[]
            for groupid in group:
                Natom=len(data[strain_id[groupid]]['structure'].species)
                magnitudes.append(data[strain_id[groupid]]['metadata']['magnitude'])
                Energies.append(data[strain_id[groupid]]['energy']/Natom)
                Energies_pred.append(data[strain_id[groupid]]['2021']['energy']/Natom)
            magnitudes=np.array(magnitudes)
            Energies=np.array(Energies)
            Energies_pred=np.array(Energies_pred)
            order=magnitudes.argsort()
            plt.scatter(magnitudes[order],Energies[order]-bd,c=colors[i],marker=markers[i],s=2,label=np.round(data[strain_id[group[0]]]['metadata']['special_direction'],4))
            plt.plot(magnitudes[order],Energies_pred[order]-b1,c=colors[i])
            plt.legend(bbox_to_anchor=(0.5, -0.05))
            i+=1
        plt.savefig("Strains_"+proto+".png",dpi=200)
        plt.close()   

    

In [7]:
thresholds=[1e-5,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14,1e-15,1e-16]
for thres in thresholds:
    print(np.sum(abs(np.dot(Es,Es.T)-1)<thres))

420572
420572
420572
420572
419672
418772
418772
418772
418772
418772
418772
236421
